In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import csv
import random

In [42]:
#读入csv中的数据，（导入检索式）
file_path = r'C:\Users\rk\Desktop\sport2020.csv'
with open(file_path) as f:
    csv_reader=csv.reader(f)
    a=list(csv_reader)
del a[0]#删除表头
c='AND DT=(Article OR Review)'
for lst in a:
    lst[3]=lst[3]+' '+c

In [23]:
a[0]

['L030-',
 'Sport0001',
 'Deakin University',
 'AD=(Deakin Univ same (exercise or Phys Act) same Australia) AND DT=(Article OR Review)']

In [24]:
##########################################################
'''若再次下载数据，将校名用代码替代，因为校名中有希腊，西班牙，葡萄牙等语言，保存非utf-8格式的TXT校名会出错'''

'若再次下载数据，将校名用代码替代，因为校名中有希腊，西班牙，葡萄牙等语言，保存非utf-8格式的TXT校名会出错'

In [25]:
a[1]

['L064-',
 'Sport0002',
 'Loughborough University',
 'AD=((Loughborough Univ or Univ Loughborough) same sport* same England) AND DT=(Article OR Review)']

In [7]:
#######################################服务器崩溃时从此往下执行#############################################################

In [6]:
browser = webdriver.Chrome('C:/Users/rk/Desktop/chromedriver.exe') #锁定浏览器
browser.get("http://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?product=WOS&search_mode=AdvancedSearch&replaceSetId=&goToPageLoc=SearchHistoryTableBanner&SID=7A2xX9mQEjWI4bS3P4J&errorQid=1#SearchHistoryTableBanner")

In [7]:
##################################服务器崩溃后，打开上面页面，重新选择之前的选项#########################################

In [26]:
def delete(history_record):
    '''删除记录，满足history_record条即删掉除第一条之外的记录'''
    historySetNum=int(browser.find_element_by_class_name('historySetNum').text[2:])
    if historySetNum==history_record or historySetNum==history_record+1:#删除记录条数
        browser.find_element_by_xpath('/html/body/div[13]/form/table/tbody/tr[1]/th[6]/div[2]/button[1]').click()#全选
        time.sleep(0.3)
        browser.find_element_by_xpath("//input[@type='checkbox'and @name='dSet']").click()#取消第一个
        time.sleep(0.4)
        browser.find_element_by_xpath('/html/body/div[13]/form/table/tbody/tr[1]/th[6]/div[2]/button[2]/i').click()#删除
        time.sleep(0.5)

In [27]:
def search(item): 
    '''向搜索框里输入检索式'''
    browser.find_element_by_id('value(input1)').clear()#清除文本框里的内容
    browser.find_element_by_id('value(input1)').send_keys(item)#往文本框里输入搜索文本（输入检索式）
    time.sleep(0.5)
    try:
        browser.find_element_by_id('searchButton').click()#点击搜素按钮
    except:
        browser.find_element_by_id('search-button').click()#点击搜素按钮

In [28]:
def write_record():
    '''保存每次检索结果，防止程序崩溃，浏览器崩溃，服务器奔溃'''
    with (open(r'C:\Users\rk\Desktop\papers20200707.txt','a+',encoding='utf-8')) as file:
            string='、'.join(item)
            file.write(string+"\n")
            #print(item)

In [46]:
count=0
b=[]

In [14]:
count=355#重启程序后设置

In [47]:
print(count)
for item in a[count:]:
    try:
        delete(20)
    except:
        pass
    
    search(item[3])
    
    d=browser.find_element_by_xpath('//*[@id="client_error_input_message"]').text#定位异常信息
        # 判断检索式是否异常，获取论文数
    if len(d)==0:#判断是否异常（如果正常运行，则不会弹出异常，那么异常信息len为零）
        v=browser.find_element_by_class_name('historyResults').text
             #如果没有异常，则定位第一个数据，因为每搜索一次，数据系列就增加一次，无法用ID定位，只能广泛定位，自动匹配第一个数据
        item.append(v)#将找到的数据拼接到原数据的后面
        write_record()
    else: 
        b.append(item)#保存异常检索式
        item.append('NA')#搜索异常的时候讲自定义的异常标识拼接到原数据后
        write_record()
        count+=1#循环+1  
        print(item)
        print("something is wrong...")
        print(count)
        continue
        #删除历史记录，以防超过200个
        
        
    delete(20)
        
    browser.find_element_by_class_name('historyResults').click()
    time.sleep(random.random()*2+2)
    x=v.replace(',','')#去掉字符串中的逗号
    i=0
    n=1
    while True:
        num_start = i*500+1
        num_end = min((i+1)*500,int(x))
        if num_start > int(x):
            break
        time.sleep(random.random()+0.3)
        
        while True:
            try:
                textexport=browser.find_element_by_xpath('//*[@id="page"]/div[1]/div[26]/div[2]/div/div/div/div[2]/div[3]/div[3]/div[2]/div[1]/button')
                if textexport.text=='导出为其他文件格式':
                    textexport.click()
                else:
                    browser.find_element_by_xpath('//*[@id="exportTypeName"]').click()#点击导出
                    browser.find_element_by_xpath('//*[@id="saveToMenu"]/li[4]/a').click()#点击其他格式
                break
            except:
                time.sleep(0.5)
                pass
            
   
        #选择记录范围按钮，点击checkbox
        time.sleep(random.random()+0.5)
        browser.find_element_by_xpath('//*[@id="numberOfRecordsRange"]').click()
        time.sleep(random.random()+0.5)
    
        elem_input = browser.find_element_by_xpath('//*[@id="markFrom"]')#输入记录范围开始数字
        elem_input.clear()
        elem_input.send_keys(num_start)
        
        #输入记录范围结束数字
        elem_input = browser.find_element_by_xpath('//*[@id="markTo"]')
        elem_input.clear()
        elem_input.send_keys(num_end)
             
        browser.find_elements_by_xpath("//div[@class='quick-output-section']//b")[0].click()#点击弹出窗口中的下拉菜单1
        time.sleep(random.random()+0.3)
        browser.find_elements_by_xpath("//span[@class='select2-results']//*[contains(text(), '全记录')]")[0].click()#选择“全记录”
        time.sleep(random.random()+0.5)
        browser.find_elements_by_xpath("//div[@class='quick-output-section']//b")[1].click()#点击弹出窗口中的下拉菜单2
        time.sleep(random.random()+0.7)
        #选择“制表符分隔（win，utf-8）
        browser.find_elements_by_xpath("//span[@class='select2-results']//*[contains(text(), '制表符')]")[2].click()
        time.sleep(random.random()*2+0.3)
        browser.find_element_by_xpath('//*[@id="exportButton"]').click()
        #等待
        time.sleep(random.random()*2+1)
        #关闭弹出窗口
        
        '''关闭弹窗'''
        while True:   #关闭弹出窗口
            if 'opacity' in browser.find_element_by_xpath('//*[@id="page"]/div[11]').get_attribute('style'):
                time.sleep(0.5)
            else:
                break
     
        i+=1 
        coun=str(n)#将数值转化为字符串
        fil='C:/Users/rk/Downloads/'+item[0]+coun+'.txt'#文件名及路径
        #time.sleep(5)#等待下载结束
        while True:
            time.sleep(0.5)#等待改名
            try:
                os.rename('C:/Users/rk/Downloads/savedrecs.txt',fil)#对下载的文件改名
            except:
                print("waiting...")
            else:
                break
        n+=1#文件序号+1
    count+=1#循环+1  
    #print(item)   
    print(count,'--',time.ctime())
    
    while True:
        try:
            browser.find_element_by_xpath('//*[@id="skip-to-navigation"]/ul[1]/li/a').click()#点击检索
            break
        except:
            time.sleep(0.3)
            pass
    
    #####################################################
    #文件名字按照数字序号排列

0
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
1
2
3
waiting...
4
waiting...
5
waiting...
6
7
8
9
10
11
12
13
14
15
waiting...
16
17
18
waiting...
19
20
waiting...
21
22
waiting...
23
24
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
waiting...
40
41
42
waiting...
waiting...
43
44
45
46
47
48
49
50
51
52
53
54
55
waiting...
56
57
waiting...
waiting...
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiti

170
171
172
173
174
175
176
177
waiting...
waiting...
178
179
180
181
182
183
184
185
186
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
187
188
189
190
191
192
193
waiting...
waiting...
waiting...
waiting...
194
195
196
197
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
198
199
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
200
201
waiting...
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
waiting...
waiting...
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
waiting...
waiting...
245
246
247
248
waiting...
waiting...
waiting...
waiting...
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
waiting...
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
wai

In [37]:
count

5

In [38]:
a[count]

['L063-',
 'Sport0006',
 'Liverpool John Moores University',
 'AD=((Liverpool John Moores Univ or Liverpool JMU) same sport* same England) AND DT=(Article OR Review)',
 '813']